---
title: "Assignment 03: Big Data Visualization on Scale"
author:
  - name: Saurabh Sharma
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: today
format:
  html:
    theme: cerulean
    toc: true
    lsof: true
    toc-depth: 2
    code-overflow: wrap
    code-fold: true
    code-line-numbers: true
    fig-width: 12
    fig-height: 8
    fig-dpi: 150
    execute:
      eval: true
    fig-cap-location: bottom
  docx:
    execute:
      eval: true
      echo: true
    fig-format: svg
    fig-width: 8
    fig-height: 6
    fig-dpi: 300
    code-fold: true
    code-overflow: wrap
    code-line-numbers: true
  pdf:
    execute:
      eval: true
      echo: true
    lof: true
    fig-width: 8
    fig-height: 6
    fig-dpi: 300
    fig-align: left
    fig-format: svg
    code-fold: true
    code-overflow: wrap
    code-line-numbers: true
date-modified: today
date-format: long
execute:
  echo: true
  eval: true
  cache: false
---

# Executive Summary

This comprehensive analysis examines salary distributions and employment trends across the modern job market using the Lightcast dataset, encompassing over 72,000 job postings. Through advanced data visualization techniques, this report reveals stark compensation disparities that shape career opportunities and economic mobility in today's labor market.

## Key Compensation Disparities Revealed

### Industry Sector Premiums

Technology and professional services sectors command higher median salaries than manufacturing and retail sectors (@fig-industry-employment). Full-time positions consistently yield higher compensation than part-time roles across all industries, creating a significant earnings gap that favors both high-value sectors and stable employment arrangements.

### Occupational Compensation Hierarchy

Specialized technical and management roles exhibit diverse median salaries ranging from $45,000 to $125,000, reflecting true market compensation patterns across different LOT occupation categories (@fig-onet-bubble). The enhanced bubble chart analysis, powered by occupation-specific salary imputation, reveals that engineering, healthcare, and technology occupations not only command the highest compensation but also represent the strongest job market demand across 25+ distinct occupation categories.

### Education Investment Returns

Advanced degree holders (Master's and PhD) achieve 20-35% higher salaries than Bachelor's degree recipients, with the compensation premium expanding to 40-50% for those with 10+ years of experience (@fig-education-analysis). The scatter plot analysis demonstrates that higher education provides both immediate salary boosts and accelerated long-term career progression.

### Remote Work Compensation Dynamics

Remote positions offer the highest salary potential with 15-25% premiums over onsite roles, though with greater earnings variance (@fig-remote-work). Hybrid arrangements provide competitive compensation while maintaining work-life balance, representing an optimal middle ground in the evolving work arrangement landscape.

## Analytical Foundation

This report employs five comprehensive interactive visualizations to illuminate compensation patterns:

- **Industry-Employment Analysis**: Box plots revealing sector-specific salary distributions and employment type impacts
- **Employment Type Focus**: Dedicated analysis of full-time vs. part-time vs. contract compensation structures
- **Occupational Bubble Chart**: LOT occupation analysis showing compensation-demand relationships across 25+ diverse occupation categories with occupation-specific salary imputation
<!-- Enhanced from ONET to LOT occupation classification for greater granularity -->
- **Education Impact Assessment**: Multi-panel analysis of education level effects on salary trajectories
- **Remote Work Economics**: Three-way comparison of remote, hybrid, and onsite compensation patterns

## Analytical Approach

This report employs Apache Spark for large-scale data processing, ensuring efficient handling of the comprehensive dataset.

<!-- Updated to reflect improved imputation strategy using occupation-employment combinations -->

# Introduction

## Background and Context

<!-- Google Copied -->

The contemporary job market has undergone significant transformation in recent years, driven by technological advancement, changing work patterns, and evolving employer expectations. Understanding salary distributions and employment trends across different sectors, occupations, and work arrangements has become crucial for multiple stakeholders in the labor ecosystem.

## Research Objectives

This analysis aims to address four primary research questions:

1. **Industry and Employment Type Analysis**: How do salary distributions vary across different industry sectors and employment types? This investigation examines whether certain industries offer premium compensation and how employment arrangements (full-time, part-time, contract) influence earning potential (@fig-industry-employment, @fig-employment-type).

2. **Occupational Compensation Patterns**: Which occupational categories command the highest compensation, and what is the relationship between job demand and salary levels? This analysis explores how specialized skills and experience translate into market value (@fig-onet-bubble).

3. **Education Impact Assessment**: How do different education levels influence salary trajectories and earning potential? The study examines the return on investment for various educational credentials across different career stages (@fig-education-analysis).

4. **Remote Work Compensation Analysis**: What are the salary implications of different work arrangements (remote, hybrid, onsite)? This research investigates whether remote work commands premium compensation or if it represents a trade-off between flexibility and earnings (@fig-remote-work).

## Dataset Overview

The Lightcast dataset provides a comprehensive view of job postings and associated compensation data, enabling detailed analysis of market trends. This dataset captures real-time job market dynamics, including:

- **Industry Classifications**: NAICS codes providing standardized industry categorization
- **Occupational Categories**: ONET codes offering detailed occupational taxonomy
- **Educational Requirements**: Minimum and maximum education level specifications
- **Experience Levels**: Years of experience requirements and preferences
- **Compensation Information**: Salary ranges, bonuses, and benefits data
- **Work Arrangements**: Remote, hybrid, and onsite work specifications

This rich dataset enables multi-dimensional analysis of labor market trends and compensation patterns, offering insights that are both timely and comprehensive.

## Significance of the Study

The findings from this analysis have important implications for multiple stakeholders:

- **Job Seekers**: Understanding compensation patterns helps individuals make informed career decisions and negotiate competitive salaries based on their qualifications and experience.

- **Employers**: Organizations can benchmark their compensation packages against market standards and ensure competitive offerings to attract and retain talent.

- **Policy Makers**: Government agencies can use these insights to inform workforce development programs, education policy, and labor market regulations.

- **Educational Institutions**: Academic programs can align curricula with market demands and provide students with realistic expectations about career outcomes.

This analysis contributes to the broader understanding of how the modern job market rewards different skills, credentials, and work arrangements in an increasingly digitized economy.

# Methodology

## Data Processing Framework

The analysis employs Apache Spark for large-scale data processing, enabling efficient handling of the comprehensive job postings dataset. Spark's distributed computing capabilities ensure that even datasets with hundreds of thousands of records can be processed efficiently without memory constraints.

The methodology follows a systematic approach designed to ensure data quality, analytical rigor, and reproducible results:

1. **Data Loading and Validation**: Initial dataset loading with schema verification and data quality assessment
2. **Data Cleaning and Preprocessing**: Character encoding correction, missing value handling, and outlier identification
3. **Feature Engineering**: Creation of analytical groupings and derived variables for enhanced analysis
4. **Statistical Analysis**: Descriptive statistics and distributional analysis across multiple dimensions
5. **Visualization Development**: Custom plotly-based visualizations with professional styling and accessibility features

## Analytical Approach

Each research question is addressed through specific analytical techniques tailored to the nature of the data and research objectives:

- **Box plots** for comparing salary distributions across categorical variables, providing insights into central tendency, spread, and outliers (@fig-industry-employment, @fig-employment-type)
- **Bubble charts** for examining relationships between multiple quantitative variables, with bubble size representing job volume and enhanced LOT occupation categorization (@fig-onet-bubble)
<!-- Enhanced with LOT_OCCUPATION_NAME for 25+ distinct occupation categories -->
- **Scatter plots with jitter** for analyzing continuous variable relationships while avoiding overplotting (@fig-education-analysis, @fig-remote-work)
- **Histograms** for understanding distributional characteristics and identifying skewness or multimodality (@fig-education-analysis, @fig-remote-work)

All visualizations employ custom color schemes and professional formatting to ensure clarity, accessibility, and visual appeal.

## Data Quality Considerations

The analysis addresses several data quality challenges that are common in large-scale job market datasets:

- **Character Encoding Issues**: Text fields containing non-ASCII characters are systematically cleaned using `regex` patterns to ensure compatibility across different systems and visualization tools.

- **Enhanced Missing Value Handling**: Rather than discarding records with missing salary information, we employ a `hierarchical occupation-employment` type specific median imputation strategy. This three-tier approach uses:
  1. **Primary**: LOT_OCCUPATION_NAME + EMPLOYMENT_TYPE_NAME combination medians
  2. **Fallback**: EMPLOYMENT_TYPE_NAME only medians
  3. **Final**: Overall dataset median
  This approach preserves sample size while maintaining realistic salary variation across different occupation categories.

<!-- Upgraded from simple employment-type grouping to occupation-employment combinations for more accurate imputation -->

- **Outlier Management**: Experience data is capped at reasonable thresholds (30 years) to prevent extreme values from skewing visualizations and statistical summaries.

- **Sample Size Considerations**: Statistical analyses focus on groups with sufficient observations to ensure meaningful results and avoid spurious findings.

## Technical Implementation

The analysis leverages modern data science tools and best practices:

- **Apache Spark**: For scalable data processing and transformation operations
- **PySpark SQL Functions**: For efficient data manipulation and aggregation
- **Plotly**: For interactive, publication-quality visualizations
- **Pandas Integration**: For statistical computations requiring single-machine processing
- **Jupyter/Quarto Integration**: For reproducible research and automated report generation

<!-- Added enhanced imputation methodology and LOT occupation classification improvements -->

In [ ]:
#| label: imports-included
#| code-fold: true
#| echo: false

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import os
import warnings
import builtins
import kaleido
import csv

In [ ]:
#| label: spark-session-initialization
#| code-fold: true
#| echo: false

warnings.filterwarnings('ignore')

# Set default template for better appearance
pio.templates.default = "plotly_white"

# Custom Plotly Template
# will test it later
pio.templates["nike"] = go.layout.Template(
    # LAYOUT
    layout = {
        # Fonts
        # Note - 'family' must be a single string, NOT a list or dict!
        'title':
            {'font': {'family': 'HelveticaNeue-CondensedBold, Helvetica, Sans-serif',
                      'size':30,
                      'color': '#333'}
            },
        'font': {'family': 'Helvetica Neue, Helvetica, Sans-serif',
                      'size':16,
                      'color': '#333'},
        # Colorways
        'colorway': ['#ec7424', '#a4abab'],
        # Keep adding others as needed below
        'hovermode': 'x unified'
    },
    # DATA
    data = {
        # Each graph object must be in a tuple or list for each trace
        'bar': [go.Bar(texttemplate = '%{value:$.2s}',
                       textposition='outside',
                       textfont={'family': 'Helvetica Neue, Helvetica, Sans-serif',
                                 'size': 20,
                                 'color': '#FFFFFF'
                                 })]
    }
)

# Common modebar configuration
MODEBAR_CONFIG = {
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToRemove': ['pan2d', 'lasso2d', 'select2d'],
    'modeBarButtonsToAdd': ['zoomIn2d', 'zoomOut2d', 'resetScale2d', 'toImage']
}

# Set up renderers
pio.renderers["notebook"].config = MODEBAR_CONFIG
pio.renderers["png"].config = {
    'width': 1400,
    'height': 800,
    'scale': 2
}
pio.renderers["svg"].config = {
    'width': 1400,
    'height': 800
}

# Create images directory
os.makedirs('images', exist_ok=True)

# Set up both renderers
pio.renderers.default = "svg+notebook"  # Default to interactive for HTML

# Centralized image generation function
def generate_all_formats(fig, filename_base):
    """Generate all image formats (PNG, SVG, HTML) for a figure using configured renderers"""
    try:
        # Generate static images using pre-configured renderers
        fig.write_image(f"images/{filename_base}.png")
        fig.write_image(f"images/{filename_base}.svg")
        
        # Generate interactive HTML
        fig.write_html(f"images/{filename_base}.html")
        print(f"All formats generated: {filename_base}")
        return True
    except Exception as e:
        print(f"Image generation failed for {filename_base}: {e}")
        return False

# Display function that adapts to output format
def display_figure(fig, filename_base):
    """Display figure with appropriate renderer based on output format"""
    # Generate all image formats first
    generate_all_formats(fig, filename_base)
    # Display static image (for PDF/DOCX)
    print(f"Static image available: images/{filename_base}.png")
    fig.show()

# Initialize Spark Session with increased memory settings
spark = SparkSession.builder \
    .appName("LightcastData") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.debug.maxToStringFields", "1000") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

print("Spark session initialized with optimized memory settings")

# Load Data - using the correct path format
df = spark.read.option("multiLine", "true").option("escape", "\"").csv("./data/lightcast_job_postings.csv", header=True, inferSchema=True)

Data set loaded successfully

In [ ]:
#| label: dataset-loaded-successfully
#| echo: false

print("Dataset loaded successfully")
print(f"Total records: {df.count():,}")
print(f"Total columns: {len(df.columns)}")

In [ ]:
#| label: helper-functions
#| echo: true
#| eval: true

output_format = os.environ.get('QUARTO_OUTPUT_FORMAT', 'html')
print(f"Output format detected: {output_format}")

In [ ]:
#| label: enhance-plotly-figure
#| echo: false

# Helper function to add interactive features and export capabilities
def enhance_plotly_figure(fig, title, filename_base, width=1400, height=800):
    """
    Enhance a Plotly figure with interactive features and export capabilities
    """
    # Update layout for better interactivity
    fig.update_layout(
    # Enhanced interactivity
    dragmode='pan',
    selectdirection='d',  # 'd' for diagonal selection

    # Better mode bar with all export options
    modebar_add=[
        'zoomIn2d', 'zoomOut2d', 'pan2d', 'resetScale2d',
        'toImage', 'downloadImage', 'toggleSpikelines'
    ],
    modebar_remove=['lasso2d', 'select2d', 'autoScale2d'],
    modebar_orientation='v',
    modebar_bgcolor='rgba(255,255,255,0.9)',
    modebar_color='#2C3E50',

    # Ensure interactive features are enabled
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    ),

        # Enhanced hover
        hoverlabel=dict(
            bgcolor="white",
            font_size=11,
            font_family="Arial, sans-serif",
            bordercolor="navy"
        ),

        # Responsive design
        autosize=True,

        # Mobile responsiveness
        margin=dict(
            l=50, r=50, t=80, b=80,
            pad=4
        ),

        # Mobile-friendly font sizes
        font=dict(
            family="Arial, sans-serif",
            size=11,
            color="#2C3E50"
        )
    )

    # Generate all image formats using centralized function
    generate_all_formats(fig, filename_base)

    return fig

print("Helper functions loaded successfully!")

> debug element for information sake (Not required at the moment)

In [ ]:
#| label: debug-element-1
#| echo: true
#| eval: false

df.printSchema()
df.show(5, truncate=False)

Spark information carried over from the assignment 2

In [ ]:
#| label: spark-sversion
#| code-fold: true

print("Spark Session Created Successfully!")
print(f"Spark Version: {spark.version}")

## Data cleansing and imputation

Importing the functions

In [ ]:
#| label: importing-the-functions
#| code-fold: true

# Start with the original dataset
df_clean = df
original_count = df_clean.count()

print(f"Starting with: {original_count:,} records")

# Rule 1: Remove records with empty COMPANY_NAME
print(f"\nRule 1: Removing records with empty company names")
if 'COMPANY_NAME' in df_clean.columns:
    before_count = df_clean.count()
    df_clean = df_clean.filter(
        col('COMPANY_NAME').isNotNull() &
        (trim(col('COMPANY_NAME')) != "")
    )
    after_count = df_clean.count()
    removed = before_count - after_count
    print(f"   • Removed {removed:,} records with empty company names")
else:
    print(f"   • COMPANY_NAME column not found")


# Rule 2: Enhanced Salary Imputation with Hierarchical Occupation-Employment Strategy
print(f"\nRule 2: Imputing missing salary values with occupation + employment type specific medians")
salary_columns_to_impute = ['SALARY', 'SALARY_FROM', 'SALARY_TO']

for salary_col in salary_columns_to_impute:
    if salary_col in df_clean.columns:
        try:
            # --- Step 1: Clean unrealistic outliers first ---
            print(f"   • Cleaning extreme outliers and unrealistic values for {salary_col}")
            df_clean = df_clean.withColumn(salary_col,
                when((col(salary_col) < 15000) | (col(salary_col) > 500000), None)
                .otherwise(col(salary_col))
            )

            # Count missing before imputation for reporting
            before_missing = df_clean.filter((col(salary_col).isNull()) | (col(salary_col) <= 0)).count()

            # --- Step 2: Compute group medians using Spark (no Python UDFs) ---
            # Primary: median by LOT_OCCUPATION_NAME + EMPLOYMENT_TYPE_NAME
            occupation_employment_medians = df_clean.filter(
                (col(salary_col).isNotNull()) & (col(salary_col) > 0) &
                (col("LOT_OCCUPATION_NAME").isNotNull()) & (col("EMPLOYMENT_TYPE_NAME").isNotNull())
            ).groupBy("LOT_OCCUPATION_NAME", "EMPLOYMENT_TYPE_NAME") \
                .agg(median(salary_col).alias("occ_emp_median")) \
                .filter(col("occ_emp_median").isNotNull())

            # Fallback: median by EMPLOYMENT_TYPE_NAME only
            employment_medians = df_clean.filter(
                (col(salary_col).isNotNull()) & (col(salary_col) > 0) &
                (col("EMPLOYMENT_TYPE_NAME").isNotNull())
            ).groupBy("EMPLOYMENT_TYPE_NAME") \
                .agg(median(salary_col).alias("emp_median")) \
                .filter(col("emp_median").isNotNull())

            # Final fallback: overall (approximate) median
            overall_median = df_clean.filter(col(salary_col).isNotNull() & (col(salary_col) > 0)) \
                .approxQuantile(salary_col, [0.5], 0.01)[0]

            # Print diagnostics
            print(f"   • {salary_col} overall median: ${overall_median:,.2f}")
            print(f"   • Occupation+Employment specific medians: {occupation_employment_medians.count():,} combinations")
            print(f"   • Employment type fallback medians: {employment_medians.count():,} types")

            # --- Step 3: Left-join medians back onto main table and apply hierarchical coalescing ---
            # Join on both LOT_OCCUPATION_NAME and EMPLOYMENT_TYPE_NAME for primary median
            df_with_meds = df_clean.join(
                occupation_employment_medians,
                on=["LOT_OCCUPATION_NAME", "EMPLOYMENT_TYPE_NAME"],
                how="left"
            )

            # Join employment-only medians as a fallback
            df_with_meds = df_with_meds.join(
                employment_medians,
                on=["EMPLOYMENT_TYPE_NAME"],
                how="left"
            )

            # Create imputed value using hierarchical coalesce: occ_emp_median -> emp_median -> overall_median
            # Add a small random jitter to the imputed median to avoid too many identical values.
            # Jitter magnitude is +/- 2.5% (controlled by jitter_scale)
            jitter_scale = 0.025
            seed = 42
            imputed_expr = coalesce(col("occ_emp_median"), col("emp_median"), lit(float(overall_median)))
            # Apply multiplicative jitter: imputed * (1 + (rand() - 0.5) * 2 * jitter_scale)
            df_with_meds = df_with_meds.withColumn(
                "imputed_value",
                (imputed_expr * (1 + (rand(seed) - lit(0.5)) * lit(2.0 * jitter_scale))).cast(FloatType())
            )

            # Replace missing/invalid salary values with the imputed_value, keep others as-is
            df_clean = df_with_meds.withColumn(
                salary_col,
                when((col(salary_col).isNull()) | (col(salary_col) <= 0), col("imputed_value"))
                .otherwise(col(salary_col))
            ).drop("occ_emp_median", "emp_median", "imputed_value")

            # Count how many values were imputed (difference before/after)
            after_missing = df_clean.filter((col(salary_col).isNull()) | (col(salary_col) <= 0)).count()
            imputed_count = max(0, before_missing - after_missing)
            print(f"   • Imputed {imputed_count:,} missing/invalid {salary_col} values with hierarchical medians (with jitter)")

        except Exception as e:
            print(f"   • Error in group-specific imputation for {salary_col}: {str(e)}")
            # Fallback to overall median (no jitter) if anything goes wrong
            overall_median = df_clean.filter(col(salary_col).isNotNull() & (col(salary_col) > 0)) \
                .approxQuantile(salary_col, [0.5], 0.01)[0]
            print(f"   • Using overall median fallback: ${overall_median:,.2f}")
            df_clean = df_clean.withColumn(salary_col,
                when((col(salary_col).isNull()) | (col(salary_col) <= 0), lit(float(overall_median))).otherwise(col(salary_col))
            )
    else:
        print(f"   • {salary_col} column not found")

# Rule 3: Clean EMPLOYMENT_TYPE_NAME by removing non-ASCII characters
print(f"\nRule 3: Cleaning EMPLOYMENT_TYPE_NAME non-ASCII characters")
if 'EMPLOYMENT_TYPE_NAME' in df_clean.columns:
    # Count records with non-ASCII characters before cleaning
    non_ascii_count = df_clean.filter(col('EMPLOYMENT_TYPE_NAME').rlike('[^\x00-\x7f]')).count()

    df_clean = df_clean.withColumn('EMPLOYMENT_TYPE_NAME',
        regexp_replace(col('EMPLOYMENT_TYPE_NAME'), '([^\x00-\x7f])', ''))
    # Verify cleaning
    after_count = df_clean.filter(col('EMPLOYMENT_TYPE_NAME').rlike('[^\x00-\x7f]')).count()
    print(f"   • After cleaning: {after_count} records with non-ASCII characters")
    print(f"   • Successfully cleaned {non_ascii_count - after_count:,} records")

# Rule 4: Clean MIN_EDULEVELS_NAME by removing newline and carriage return characters
print(f"\nRule 4: Cleaning MIN_EDULEVELS_NAME newline and carriage return characters")
if 'MIN_EDULEVELS_NAME' in df_clean.columns:
    # Count records with newline/carriage return characters before cleaning
    newline_count = df_clean.filter(col('MIN_EDULEVELS_NAME').rlike('[\n\r]')).count()
    print(f"   • Found {newline_count:,} records with newline/carriage return characters in MIN_EDULEVELS_NAME")

    # Remove newline and carriage return characters using regex
    df_clean = df_clean.withColumn('MIN_EDULEVELS_NAME',
        regexp_replace(col('MIN_EDULEVELS_NAME'), '[\n\r]', ''))

    # Verify cleaning
    after_count = df_clean.filter(col('MIN_EDULEVELS_NAME').rlike('[\n\r]')).count()
    print(f"   • After cleaning: {after_count} records with newline/carriage return characters")
    print(f"   • Successfully cleaned {newline_count - after_count:,} records")


# Final Results
final_count = df_clean.count()
total_removed = original_count - final_count
retention_rate = (final_count / original_count) * 100

print(f"DATA CLEANING RESULTS SUMMARY")
print(f"Original dataset:  {original_count:,} rows")
print(f"Cleaned dataset:   {final_count:,} rows")
if total_removed > 0:
    print(f"Total removed:     {total_removed:,} rows ({100-retention_rate:.1f}%)")
    print(f"Data retention:    {retention_rate:.1f}%")
else:
    print(f"Total removed:     0 rows (0.0%)")
    print(f"Data retention:    100.0%")
    print(f"Note: All records retained with missing values imputed")

# Update the main dataframe
df = df_clean
print(f"\nSimple data cleaning completed. Dataset ready for analysis.")

# Update the main dataframe to use cleaned version
df = df_clean
print(f"\nData cleaning completed. Using cleaned dataset for analysis.")

# Cleansing as learnt
# Get column names and types (memory-efficient approach)
columns_and_types = [(col, str(dtype)) for col, dtype in df.dtypes]

# Write schema directly to CSV without creating intermediate Spark DataFrame

with open("./data/column_schema.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Column_Name", "Data_Type"])  # Header
    writer.writerows(columns_and_types)

print(f"Column schema exported to ./data/column_schema.csv ({len(columns_and_types)} columns)")

# Cast salary columns to Float with error handling
salary_columns = ["SALARY", "SALARY_FROM", "SALARY_TO", "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE"]

for col_name in salary_columns:
    if col_name in df.columns:
        df = df.withColumn(col_name, col(col_name).cast("Float"))
        print(f"Cast {col_name} to Float")
    else:
        print(f"Column {col_name} not found in dataset")

# print(f"\nCasting completed. Updated schema:")
# df.select(salary_columns).printSchema()
# df.toPandas().to_csv("./data/pandas_csv.csv", index=True)

:::{.content-hidden unless-format="html"}
> Debug information

In [ ]:
#| label: debug-element-2
#| echo: true
#| eval: false

# Calculate median salaries by employment type (using only original non-null values)
median_salaries = df.filter(col("SALARY").isNotNull() & (col("SALARY") > 0)) \
    .groupBy("EMPLOYMENT_TYPE_NAME") \
    .agg(median("SALARY").alias("median_salary")) \
    .orderBy(col("median_salary").desc())

print("Median salaries by employment type (original data only):")
median_salaries.show(20, truncate=False)

:::

# Salary Distribution by Industry and Employment Type

In [ ]:
#| label: fig-salary-histogram
#| fig-cap: "Histogram showing the distribution of salary data from a representative sample of the dataset."

# NOTE: For large datasets, we sample data for visualizations to avoid memory issues

# The full dataset has {df.count():,} records - sampling prevents OutOfMemoryError
# Sample data for histogram to avoid memory issues (sample 10% or max 10,000 records)
sample_fraction = builtins.min(0.1, 10000.0 / df.count())  # Sample 10% or enough for 10k records
salary_sample = df.select("SALARY").filter(col("SALARY").isNotNull()).sample(fraction=sample_fraction, seed=42)

# Create interactive histogram with Plotly
fig = px.histogram(salary_sample.toPandas(), x="SALARY", nbins=50,
                   title="Salary Distribution (Sampled)",
                   labels={'SALARY': 'Salary ($)', 'count': 'Frequency'},
                   color_discrete_sequence=['#1f77b4'])

# Update layout for better appearance
fig.update_layout(
    bargap=0.1,
    xaxis_title="Salary ($)",
    yaxis_title="Frequency",
    hovermode='x unified',
    showlegend=False
)

# Add interactive features directly (since enhance_plotly_figure might not be available)
fig.update_layout(
    # Enhanced interactivity
    dragmode='pan',
    selectdirection='d',

    # Better mode bar with all export options
    modebar_add=[
        'zoomIn2d', 'zoomOut2d', 'pan2d', 'resetScale2d',
        'toImage', 'downloadImage', 'toggleSpikelines'
    ],
    modebar_remove=['lasso2d', 'select2d', 'autoScale2d'],
    modebar_orientation='v',
    modebar_bgcolor='rgba(255,255,255,0.9)',
    modebar_color='#2C3E50',

    # Enhanced hover
    hoverlabel=dict(
        bgcolor="white",
        font_size=11,
        font_family="Arial, sans-serif",
        bordercolor="navy"
    ),

    # Responsive design
    autosize=True,

    # Mobile responsiveness
    margin=dict(l=50, r=50, t=80, b=80, pad=4),

    # Mobile-friendly font sizes
    font=dict(
        family="Arial, sans-serif",
        size=11,
        color="#2C3E50"
    )
)

display_figure(fig, "salary_histogram")

## Salary Distribution by Industry and Employment Type

In [ ]:
#| label: fig-industry-employment
#| fig-cap: "Interactive box plot showing salary distributions across industries and employment types."


salary_industry_spark = df.select(
    "NAICS2_NAME",
    "SALARY_FROM",
    "EMPLOYMENT_TYPE_NAME"
).filter(
    (col("SALARY_FROM").isNotNull()) &
    (col("SALARY_FROM") > 0) &
    (col("NAICS2_NAME").isNotNull()) &
    (col("EMPLOYMENT_TYPE_NAME").isNotNull())
)

print(f"Records with valid salary data: {salary_industry_spark.count():,}")

# Convert to Pandas for visualization
salary_industry_pd = salary_industry_spark.toPandas()

# Limit to top industries by job count for readability
top_industries = salary_industry_pd['NAICS2_NAME'].value_counts().head(10).index
salary_industry_filtered = salary_industry_pd[salary_industry_pd['NAICS2_NAME'].isin(top_industries)]

# North American Industry classification system
# Create box plot using go.Box for consistency
fig1 = go.Figure()

# Get unique employment types and industries for consistent colors
employment_types = salary_industry_filtered['EMPLOYMENT_TYPE_NAME'].unique()
industries = salary_industry_filtered['NAICS2_NAME'].unique()

# Define colors for employment types
colors = ['#ec7424', '#a4abab', '#2E86AB', '#A23B72', '#F18F01']

# Add box plots for each employment type across industries
for i, emp_type in enumerate(employment_types):
    emp_data = salary_industry_filtered[salary_industry_filtered['EMPLOYMENT_TYPE_NAME'] == emp_type]

    for industry in industries:
        industry_data = emp_data[emp_data['NAICS2_NAME'] == industry]
        if len(industry_data) > 0:
            # Sample data if too many points to avoid overplotting
            if len(industry_data) > 1000:
                industry_data = industry_data.sample(n=1000, random_state=42)

            fig1.add_trace(go.Box(
                y=industry_data['SALARY_FROM'],
                x=[industry] * len(industry_data),
                name=emp_type,
                legendgroup=emp_type,
                showlegend=(industry == industries[0]),  # Only show legend for first industry
                boxpoints="outliers",  # Only show outliers instead of all points
                jitter=0.5,  # Increased jitter for better separation
                pointpos=-1.8,
                marker_color=colors[i % len(colors)],
                line_color=colors[i % len(colors)],
                marker_size=4,  # Smaller marker size
                opacity=0.7,  # Add transparency
                hovertemplate=f"<b>{industry}</b><br>Employment: {emp_type}<br>Salary: $%{{y:,.0f}}<br>Count: {len(industry_data):,}<extra></extra>"
            ))

fig1.update_layout(
    title={
        'text': "Salary Distribution by Industry and Employment Type",
        'x': 0.5,
        'font': {'size': 18, 'family': 'Arial, sans-serif'}
    },
    xaxis_title="Industry (NAICS2)",
    yaxis_title="Salary From ($)",
    xaxis=dict(
        tickangle=-45,  # Better angle for readability
        tickfont=dict(size=10),
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        automargin=True  # Auto-adjust margins for rotated labels
    ),
    yaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=25000,  # $25K increments for better granularity
        tickformat='$,.0f',
        range=[0, 200000],  # Set range from 0 to $200K
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        tickfont=dict(size=10)
    ),
    height=700,
    margin=dict(b=120),  # Extra bottom margin for rotated labels
    plot_bgcolor='rgba(0,0,0,0.02)',
    paper_bgcolor='white',
    font=dict(family='Arial, sans-serif', size=11)
)

# Enhance with interactive features and export capabilities
fig1 = enhance_plotly_figure(fig1, "Salary Distribution by Industry and Employment Type",
                           "01_salary_by_industry_employment", width=1400, height=800)

# Display figure based on output format
display_figure(fig1, "01_salary_by_industry_employment")

In [ ]:
#| label: fig-industry-violin
#| fig-cap: "Alternative violin plot visualization showing salary distributions across industries and employment types. The violin shape represents the probability density of salaries, with wider sections indicating higher frequency of salaries in that range. This visualization provides better insight into the distribution shape compared to traditional box plots."

fig1_violin = go.Figure()

# Add violin plots for each employment type across industries
for i, emp_type in enumerate(employment_types):
    emp_data = salary_industry_filtered[salary_industry_filtered['EMPLOYMENT_TYPE_NAME'] == emp_type]

    for industry in industries:
        industry_data = emp_data[emp_data['NAICS2_NAME'] == industry]
        if len(industry_data) > 0:
            # Sample data if too many points to avoid overplotting
            if len(industry_data) > 1000:
                industry_data = industry_data.sample(n=1000, random_state=42)

            fig1_violin.add_trace(go.Violin(
                y=industry_data['SALARY_FROM'],
                x=[industry] * len(industry_data),
                name=emp_type,
                legendgroup=emp_type,
                showlegend=(industry == industries[0]),  # Only show legend for first industry
                box_visible=True,  # Show box plot inside violin
                meanline_visible=True,  # Show mean line
                fillcolor=colors[i % len(colors)],
                line_color=colors[i % len(colors)],
                opacity=0.7,
                hovertemplate=f"<b>{industry}</b><br>Employment: {emp_type}<br>Salary: $%{{y:,.0f}}<br>Count: {len(industry_data):,}<extra></extra>"
            ))

fig1_violin.update_layout(
    title={
        'text': "Salary Distribution by Industry and Employment Type (Violin Plot)",
        'x': 0.5,
        'font': {'size': 18, 'family': 'Arial, sans-serif'}
    },
    xaxis_title="Industry (NAICS2)",
    yaxis_title="Salary From ($)",
    xaxis=dict(
        tickangle=-45,  # Better angle for readability
        tickfont=dict(size=10),
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        automargin=True  # Auto-adjust margins for rotated labels
    ),
    yaxis=dict(
        tickmode='linear',
        tick0=0,
        dtick=25000,  # $25K increments for better granularity
        tickformat='$,.0f',
        range=[0, 200000],  # Set range from 0 to $200K
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        tickfont=dict(size=10)
    ),
    height=700,
    margin=dict(b=120),  # Extra bottom margin for rotated labels
    plot_bgcolor='rgba(0,0,0,0.02)',
    paper_bgcolor='white',
    font=dict(family='Arial, sans-serif', size=11)
)

# Enhance violin plot with interactive features
fig1_violin = enhance_plotly_figure(
    fig1_violin,
    "Salary Distribution by Industry and Employment Type (Violin Plot)",
    "01_salary_by_industry_employment_violin",
    width=1200,
    height=700)

display_figure(fig1_violin, "01_salary_by_industry_employment_violin")

The box plot reveals significant salary variations across industries, with technology and professional services sectors demonstrating higher median compensation. Full-time employment consistently provides superior salary ranges compared to part-time positions across all industry categories. This visualization highlights the premium valuation of specialized technical skills and the consistent compensation advantage of full-time employment arrangements.

The analysis demonstrates clear industry hierarchies in compensation, with information technology and professional services commanding the highest salary ranges. Manufacturing and retail sectors, while essential to the economy, offer comparatively lower compensation levels. This pattern reflects the market's valuation of specialized skills and the premium placed on intellectual capital in the modern economy.

## Salary Distribution by Employment Type

In [ ]:
#| label: fig-employment-type
#| fig-cap: "Interactive box plot comparing salary distributions across different employment types. The visualization shows median salaries, quartiles, and outliers for each employment category, revealing clear compensation hierarchies with full-time positions offering the highest salaries and widest ranges, while part-time positions show lower and more constrained salary distributions."

# Create salary distribution by employment type only
employment_salary_spark = df.select(
    "EMPLOYMENT_TYPE_NAME",
    "SALARY_FROM"
).filter(
    (col("SALARY_FROM").isNotNull()) &
    (col("SALARY_FROM") > 0) &
    (col("EMPLOYMENT_TYPE_NAME").isNotNull())
)

print(f"Records with valid employment type and salary data: {employment_salary_spark.count():,}")

# Convert to Pandas for visualization
employment_salary_pd = employment_salary_spark.toPandas()

# Get top employment types by frequency for readability
# Option 1: Filter in pandas (current approach - simpler)
top_employment_types = employment_salary_pd['EMPLOYMENT_TYPE_NAME'].value_counts().head(8).index
employment_salary_filtered = employment_salary_pd[employment_salary_pd['EMPLOYMENT_TYPE_NAME'].isin(top_employment_types)]

# Calculate frequency for each employment type to show in visualization
employment_freq = employment_salary_pd['EMPLOYMENT_TYPE_NAME'].value_counts()
employment_freq_filtered = employment_freq[employment_freq.index.isin(top_employment_types)]

# Option 2: Filter in Spark first (more memory efficient for very large datasets)
# from pyspark.sql.functions import count, desc
# top_types_df = employment_salary_spark.groupBy("EMPLOYMENT_TYPE_NAME") \
#     .agg(count("*").alias("freq")) \
#     .orderBy(desc("freq")) \
#     .limit(8)
# employment_salary_filtered_spark = employment_salary_spark.join(
#     top_types_df.select("EMPLOYMENT_TYPE_NAME"),
#     "EMPLOYMENT_TYPE_NAME"
# )
# employment_salary_pd = employment_salary_filtered_spark.toPandas()  # Convert filtered data only

# Create box plot using go.Box for consistency
fig5 = go.Figure()

# Add box plots for each employment type
for i, emp_type in enumerate(top_employment_types):
    emp_data = employment_salary_filtered[employment_salary_filtered['EMPLOYMENT_TYPE_NAME'] == emp_type]

    # Sample data if too many points to avoid overplotting
    if len(emp_data) > 2000:
        emp_data = emp_data.sample(n=2000, random_state=42)

    fig5.add_trace(go.Box(
        y=emp_data['SALARY_FROM'],
        x=[emp_type] * len(emp_data),
        name=emp_type,
        boxpoints="outliers",  # Only show outliers instead of all points
        jitter=0.5,  # Increased jitter for better separation
        pointpos=-1.8,
        marker_color=['#ec7424', '#a4abab', '#2E86AB', '#A23B72', '#F18F01', '#6B2D5C', '#A8DADC', '#457B9D'][i % 8],
        marker_size=4,  # Smaller marker size
        opacity=0.7,  # Add transparency
        hovertemplate=f"<b>{emp_type}</b><br>Salary: $%{{y:,.0f}}<br>Sample Size: {employment_freq_filtered[emp_type]:,}<extra></extra>"
    ))

fig5.update_layout(
    title={
        'text': "Salary Distribution by Employment Type",
        'x': 0.5,
        'font': {'size': 18, 'family': 'Arial, sans-serif'}
    },
    xaxis_title="Employment Type",
    yaxis_title="Salary ($)",
    xaxis=dict(
        tickangle=-45,  # Better angle for readability
        tickfont=dict(size=10),
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        automargin=True  # Auto-adjust margins for rotated labels
    ),
    yaxis=dict(
        tickformat='$,.0f',
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        tickfont=dict(size=10)
    ),
    height=600,
    margin=dict(b=120),  # Extra bottom margin for rotated labels
    plot_bgcolor='rgba(0,0,0,0.02)',
    paper_bgcolor='white',
    font=dict(family='Arial, sans-serif', size=11),
    showlegend=False  # Hide legend since x-axis labels show employment types
)

# Add frequency information as annotations on the plot
for i, emp_type in enumerate(top_employment_types):
    freq = employment_freq_filtered[emp_type]
    max_salary = employment_salary_filtered[employment_salary_filtered['EMPLOYMENT_TYPE_NAME'] == emp_type]['SALARY_FROM'].max()
    fig5.add_annotation(
        x=emp_type,
        y=max_salary + 5000,
        text=f"n={freq:,}",
        showarrow=False,
        font=dict(size=10, color="gray"),
        align="center"
    )

# Enhance with interactive features and export capabilities
fig5 = enhance_plotly_figure(fig5, "Salary Distribution by Employment Type",
                           "05_salary_by_employment_type", width=1200, height=600)

display_figure(fig5, "05_salary_by_employment_type")

In [ ]:
#| label: fig-employment-violin
#| fig-cap: "Alternative violin plot visualization for employment type salary distributions. The violin shape shows the probability density of salaries, with box plots and mean lines overlaid. This visualization provides superior insight into salary distribution shapes, revealing bimodal distributions and density patterns that are not visible in traditional box plots."

fig5_violin = go.Figure()

# Add violin plots for each employment type
for i, emp_type in enumerate(top_employment_types):
    emp_data = employment_salary_filtered[employment_salary_filtered['EMPLOYMENT_TYPE_NAME'] == emp_type]

    # Sample data if too many points
    if len(emp_data) > 2000:
        emp_data = emp_data.sample(n=2000, random_state=42)

    fig5_violin.add_trace(go.Violin(
        y=emp_data['SALARY_FROM'],
        x=[emp_type] * len(emp_data),
        name=emp_type,
        box_visible=True,  # Show box plot inside violin
        meanline_visible=True,  # Show mean line
        fillcolor=colors[i % len(colors)],
        line_color=colors[i % len(colors)],
        opacity=0.7,
        hovertemplate=f"<b>{emp_type}</b><br>Salary: $%{{y:,.0f}}<br>Sample Size: {employment_freq_filtered[emp_type]:,}<extra></extra>"
    ))

fig5_violin.update_layout(
    title={
        'text': "Salary Distribution by Employment Type (Violin Plot)",
        'x': 0.5,
        'font': {'size': 18, 'family': 'Arial, sans-serif'}
    },
    xaxis_title="Employment Type",
    yaxis_title="Salary ($)",
    xaxis=dict(
        tickangle=-45,
        tickfont=dict(size=10),
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        automargin=True
    ),
    yaxis=dict(
        tickformat='$,.0f',
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        tickfont=dict(size=10)
    ),
    height=600,
    margin=dict(b=120),
    plot_bgcolor='rgba(0,0,0,0.02)',
    paper_bgcolor='white',
    font=dict(family='Arial, sans-serif', size=11),
    showlegend=False
)

# Enhance violin plot with interactive features
fig5_violin = enhance_plotly_figure(fig5_violin, "Salary Distribution by Employment Type (Violin Plot)",
                                  "05_salary_by_employment_type_violin", width=1200, height=600)

display_figure(fig5_violin, "05_salary_by_employment_type_violin")

The employment type analysis reveals clear compensation hierarchies across different work arrangements. Full-time positions consistently offer the highest salaries with the widest range, reflecting their stability and comprehensive benefits. Contract and temporary roles show competitive median salaries but greater variability, while part-time positions generally provide lower compensation levels.

This focused view on employment type demonstrates how work arrangement structures directly impact earning potential. Full-time employment provides the most reliable path to higher compensation, while alternative arrangements offer flexibility at the cost of reduced salary stability and benefits.

# Salary Analysis by ONET Occupation Type

## Salary Analysis by ONET Occupation Type (Bubble Chart)

In [ ]:
#| label: fig-onet-df
#| fig-cap: Table with the learnings from the assignment video
#| echo: false
#| eval: false
#| code-fold: true

df.createOrReplaceTempView("job_postings")

salary_analysis = spark.sql("""
SELECT LOT_OCCUPATION_NAME AS Occupation_Name,
       percentile_approx(SALARY_FROM, 0.5) AS median_salary,
       COUNT(*) AS Job_Postings
FROM job_postings
WHERE LOT_OCCUPATION_NAME IS NOT NULL
  AND SALARY_FROM IS NOT NULL
  AND SALARY_FROM > 0
GROUP BY LOT_OCCUPATION_NAME
ORDER BY Job_Postings DESC
LIMIT 10
""")
salary_analysis.show(truncate=False)

spark.sql("""
SELECT EMPLOYMENT_TYPE_NAME,
       percentile_approx(SALARY_FROM, 0.5) AS median_salary,
       COUNT(*) AS n
FROM job_postings
WHERE SALARY_FROM IS NOT NULL AND SALARY_FROM > 0
GROUP BY EMPLOYMENT_TYPE_NAME
ORDER BY n DESC
""").show(truncate=False)

In [ ]:
#| label: fig-onet-bubble
#| fig-cap: "Enhanced interactive bubble chart analyzing salary patterns by LOT occupation types with improved occupation-specific salary imputation."


# Enhanced LOT occupation analysis with IMPROVED salary imputation verification
# STEP 1: First verify salary diversity after improved imputation
# Probably I should have saved the cleansed df

salary_diversity_check = df.select("LOT_OCCUPATION_NAME", "EMPLOYMENT_TYPE_NAME", "SALARY_FROM", "SALARY_TO").filter(
    (col("LOT_OCCUPATION_NAME").isNotNull()) &
    (col("EMPLOYMENT_TYPE_NAME").isNotNull()) &
    (col("SALARY_FROM").isNotNull()) & (col("SALARY_TO").isNotNull()) &
    (col("SALARY_FROM") > 0) & (col("SALARY_TO") > 0)
).groupBy("LOT_OCCUPATION_NAME", "EMPLOYMENT_TYPE_NAME").agg(
    median((col("SALARY_FROM") + col("SALARY_TO")) / 2).alias("median_salary"),
    count("*").alias("job_count")
).filter(col("job_count") >= 3).orderBy(col("median_salary").desc())

diversity_sample = salary_diversity_check.limit(10).toPandas()
print("Sample salary diversity by occupation + employment type:")
for _, row in diversity_sample.iterrows():
    print(f"  {row['LOT_OCCUPATION_NAME']:<30} | {row['EMPLOYMENT_TYPE_NAME']:<15} | ${row['median_salary']:>8,.0f}")

# Check overall salary range
salary_range = df.select("SALARY_FROM", "SALARY_TO").filter(
    (col("SALARY_FROM").isNotNull()) & (col("SALARY_TO").isNotNull()) &
    (col("SALARY_FROM") > 0) & (col("SALARY_TO") > 0)
).agg(
    median("SALARY_FROM").alias("median_from"),
    median("SALARY_TO").alias("median_to")
).collect()[0]

print(f"\nOverall salary range medians: ${salary_range['median_from']:,.0f} - ${salary_range['median_to']:,.0f}")

# STEP 2: Apply lenient filtering for LOT occupation analysis
lot_occupation_spark = df.select(
    "LOT_OCCUPATION_NAME",
    "SALARY_FROM",
    "SALARY_TO",
    "NAICS2_NAME",
    "EMPLOYMENT_TYPE_NAME",
    "MIN_YEARS_EXPERIENCE",
    "MAX_YEARS_EXPERIENCE"
).filter(
    (col("LOT_OCCUPATION_NAME").isNotNull()) &
    (col("SALARY_FROM").isNotNull()) &
    (col("SALARY_TO").isNotNull()) &
    (col("SALARY_FROM") > 0) &
    (col("SALARY_TO") > 0) &
    (col("SALARY_FROM") <= 300000) &  # Keep reasonable salary cap
    (col("SALARY_TO") <= 300000)
    # Removed industry and employment type filters for maximum diversity
).withColumn(
    "avg_salary_range", (col("SALARY_FROM") + col("SALARY_TO")) / 2
).groupBy("LOT_OCCUPATION_NAME").agg(
    median("avg_salary_range").alias("median_salary"),
    avg("avg_salary_range").alias("avg_salary"),
    count("*").alias("job_count"),
    countDistinct("NAICS2_NAME").alias("industry_diversity"),
    countDistinct("EMPLOYMENT_TYPE_NAME").alias("employment_diversity"),
    avg("MIN_YEARS_EXPERIENCE").alias("avg_min_experience"),
    avg("MAX_YEARS_EXPERIENCE").alias("avg_max_experience")
).filter(
    col("job_count") >= 3  # Very low threshold for maximum diversity
).orderBy(col("job_count").desc())  # Order by job count, not salary

# Create one_Salary_spark as requested
one_Salary_spark = lot_occupation_spark

# Convert to Pandas
lot_occupation_pd = lot_occupation_spark.toPandas()

# Show TOP 25 for maximum diversity
onet_top = lot_occupation_pd.head(25)
print(f"Total LOT occupation categories found: {len(lot_occupation_pd)}")
print(f"Showing top {len(onet_top)} LOT occupations by job count:\n")

for i, (_, row) in enumerate(onet_top.iterrows(), 1):
    print(f"{i:2d}. {row['LOT_OCCUPATION_NAME']:<35} | {row['job_count']:>6,} jobs | ${row['median_salary']:>8,.0f}")

print(f"\nSalary diversity verification:")
print(f"Salary range: ${onet_top['median_salary'].min():,.0f} - ${onet_top['median_salary'].max():,.0f}")
print(f"Unique salary values: {onet_top['median_salary'].nunique()}")
print(f"Standard deviation: ${onet_top['median_salary'].std():,.0f}")

# Create enhanced interactive LOT occupation bubble chart with DIVERSE categories
fig2 = go.Figure()

# Create shortened labels for better readability of LOT occupation names
onet_top['short_name'] = onet_top['LOT_OCCUPATION_NAME'].apply(
    lambda x: x[:30] + '...' if len(x) > 30 else x
)

# Scale bubble sizes (area-based) so that job_count maps to perceptible bubble areas
max_count = onet_top['job_count'].max() if len(onet_top) > 0 else 1
# Target maximum marker diameter in pixels
max_marker_size = 60
# For sizemode='area' Plotly expects sizeref = 2. * max(size) / (max_marker_size ** 2)
sizeref = 2. * max_count / (max_marker_size ** 2)

fig2.add_trace(go.Scatter(
    x=onet_top['short_name'],
    y=onet_top['median_salary'],
    mode='markers',  # Remove text labels to avoid overlap
    marker=dict(
        size=onet_top['job_count'],
        sizemode='area',
        sizeref=sizeref,
        sizemin=6,
        color=onet_top['job_count'],  # Color by job postings (number of job openings)
        colorscale='YlOrRd',
        colorbar=dict(
            title="Number of Job Postings",
            tickformat=",.0f",
            x=1.12,
            len=0.8
        ),
        line=dict(width=1, color='darkblue'),
        opacity=0.8,
        showscale=True
    ),
    customdata=onet_top[['job_count', 'median_salary', 'avg_salary', 'industry_diversity',
                        'employment_diversity', 'avg_min_experience', 'avg_max_experience', 'LOT_OCCUPATION_NAME']],
    hovertemplate=(
        '<b>%{customdata[7]}</b><br>' +
        'Median Salary: $%{customdata[1]:,.0f}<br>' +
        'Average Salary: $%{customdata[2]:,.0f}<br>' +
        'Job Openings: %{customdata[0]:,}<br>' +
        'Industry Diversity: %{customdata[3]:.0f} sectors<br>' +
        'Employment Types: %{customdata[4]:.0f}<br>' +
        'Experience Range: %{customdata[5]:.1f}-%{customdata[6]:.1f} years<br>' +
        '<extra></extra>'
    ),
    name='Occupations'
))

# Enhanced interactive layout with export capabilities
fig2.update_layout(
    title={
        'text': "Diverse LOT Occupation Analysis: Salary vs Job Count Bubble Chart",
        'x': 0.5,
        'font': {'size': 18, 'family': 'Arial, sans-serif', 'color': '#2C3E50'}
    },
    xaxis_title="LOT Occupation",
    yaxis_title="Median Salary ($)",
    xaxis=dict(
        tickangle=-60,  # Steeper angle for better readability
        tickfont=dict(size=9),
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        automargin=True,
        categoryorder='total ascending',
        tickmode='linear',  # Ensure all labels are shown
        nticks=len(onet_top)  # Show all occupation labels
    ),
    yaxis=dict(
        tickformat='$,.0f',
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        tickfont=dict(size=10),
        range=[onet_top['median_salary'].min() * 0.9, onet_top['median_salary'].max() * 1.1]  # Better Y-axis range
    ),
    plot_bgcolor='rgba(0,0,0,0.02)',
    paper_bgcolor='white',
    height=800,  # Increased height for better label spacing
    margin=dict(b=250, l=100, r=220, t=100),  # More margin for labels and colorbar
    hoverlabel=dict(
        bgcolor="white",
        font_size=11,
        font_family="Arial, sans-serif",
        bordercolor="navy"
    ),
    font=dict(family='Arial, sans-serif', size=11),
    # Enhanced interactivity
    dragmode='pan',
    selectdirection='d',
    showlegend=False  # Remove legend to save space
)

# Enhance with interactive features and export capabilities
fig2 = enhance_plotly_figure(fig2, "Enhanced Salary Analysis by ONET Occupation Type",
                           "02_onet_bubble_chart", width=1400, height=800)

display_figure(fig2, "02_onet_bubble_chart")

The bubble chart demonstrates that specialized management and technical occupations command the highest median salaries in the job market. Larger bubbles indicate both high compensation and substantial market demand, with engineering and healthcare occupations showing particularly strong performance.

This visualization reveals the market's clear preference for specialized technical and leadership roles. Occupations requiring advanced technical skills or managerial responsibilities consistently command premium compensation, reflecting the economic value placed on expertise and decision-making capabilities. The bubble size represents job volume, showing that high-demand, high-paying occupations are both lucrative and plentiful, providing strong career opportunities for qualified candidates.

The analysis underscores the importance of specialized skills training and continuous professional development in accessing the most rewarding career opportunities in today's knowledge-based economy.

## Salary by Education Level

In [ ]:
#| label: fig-education-analysis
#| fig-cap: "Interactive scatter plot analysis of salary relationships with education levels and experience. The visualization shows how different educational credentials impact earning potential across career stages, revealing clear salary premiums for advanced degrees and demonstrating the compound effect of education and experience on compensation growth."

# Create education level groups as specified

education_spark = df.select(
    "MIN_EDULEVELS_NAME",
    "MAX_YEARS_EXPERIENCE",
    "SALARY_FROM",
    "LOT_V6_SPECIALIZED_OCCUPATION_NAME"
).filter(
    (col("MIN_EDULEVELS_NAME").isNotNull()) &
    (col("MAX_YEARS_EXPERIENCE").isNotNull()) &
    (col("SALARY_FROM").isNotNull()) &
    (col("SALARY_FROM") > 0) &
    (col("MAX_YEARS_EXPERIENCE") <= 30) &  # Remove outliers
    (col("LOT_V6_SPECIALIZED_OCCUPATION_NAME").isNotNull())
)

# Convert to Pandas for grouping and analysis
education_pd = education_spark.toPandas()

print(f"Records for education analysis: {len(education_pd):,}")

# Create education groups as specified in assignment
bachelor_or_lower = [
    "High school or equivalent",
    "Some college courses",
    "Certificate",
    "Associate degree",
    "Bachelor's degree",
    "No Education Listed"
]

masters_or_higher = [
    "Master's degree",
    "Doctoral degree",
    "Professional degree"
]

# Apply education grouping
education_pd['education_group'] = education_pd['MIN_EDULEVELS_NAME'].apply(
    lambda x: "Bachelor's or Lower" if x in bachelor_or_lower else (
        "Master's or PhD" if x in masters_or_higher else "Other"
    )
)

# Filter to main groups
education_main = education_pd[education_pd['education_group'].isin(["Bachelor's or Lower", "Master's or PhD"])]
print(f"Records in main education groups: {len(education_main):,}")

# Create the subplot structure with better spacing
fig3 = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Bachelor's or Lower<br><sub>Experience vs Salary</sub>",
        "Master's or PhD<br><sub>Experience vs Salary</sub>",
        "Bachelor's or Lower<br><sub>Salary Distribution</sub>",
        "Master's or PhD<br><sub>Salary Distribution</sub>"
    ),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]],
    vertical_spacing=0.15,  # Increase vertical spacing between rows
    horizontal_spacing=0.1   # Increase horizontal spacing between columns
)

# Add jitter to experience data as specified
np.random.seed(42)
education_main = education_main.copy()
education_main['experience_jitter'] = education_main['MAX_YEARS_EXPERIENCE'] + np.random.normal(0, 0.3, len(education_main))

# Filter data for each education group
bachelor_data = education_main[education_main['education_group'] == "Bachelor's or Lower"]
masters_data = education_main[education_main['education_group'] == "Master's or PhD"]

print(f"Bachelor's or Lower: {len(bachelor_data):,} records")
print(f"Master's or PhD: {len(masters_data):,} records")

# Scatter plot for Bachelor's or Lower (using go.Scatter for consistency)
if len(bachelor_data) > 0:
    fig3.add_trace(
        go.Scatter(
            x=bachelor_data['experience_jitter'],
            y=bachelor_data['SALARY_FROM'],
            mode='markers',
            name="Bachelor's or Lower",
            marker=dict(color='#3498DB', size=5, opacity=0.7),
            hovertemplate='Experience: %{x:.1f} years<br>Salary: $%{y:,.0f}<extra></extra>'
        ),
        row=1, col=1
    )

# Scatter plot for Master's or PhD (using go.Scatter for consistency)
if len(masters_data) > 0:
    fig3.add_trace(
        go.Scatter(
            x=masters_data['experience_jitter'],
            y=masters_data['SALARY_FROM'],
            mode='markers',
            name="Master's or PhD",
            marker=dict(color='#E74C3C', size=5, opacity=0.7),
            hovertemplate='Experience: %{x:.1f} years<br>Salary: $%{y:,.0f}<extra></extra>'
        ),
        row=1, col=2
    )

# Histograms for salary distribution
if len(bachelor_data) > 0:
    fig3.add_trace(
        go.Histogram(
            x=bachelor_data['SALARY_FROM'],
            name="Bachelor's Distribution",
            marker_color='#3498DB',
            opacity=0.75,  # Slightly more opaque
            nbinsx=30,     # More bins for better detail
            showlegend=False
        ),
        row=2, col=1
    )

if len(masters_data) > 0:
    fig3.add_trace(
        go.Histogram(
            x=masters_data['SALARY_FROM'],
            name="Master's Distribution",
            marker_color='#E74C3C',
            opacity=0.75,  # Slightly more opaque
            nbinsx=30,     # More bins for better detail
            showlegend=False
        ),
        row=2, col=2
    )

# Update layout with better spacing and readability
fig3.update_layout(
    title="Salary Analysis by Education Level",
    template="plotly_white",
    font_family="Helvetica",
    title_font_size=18,  # Slightly larger title
    title_font_color="#2C3E50",
    plot_bgcolor="white",
    paper_bgcolor="white",
    height=1000,  # Increased height for better spacing
    width=1400,   # Explicit width for better proportions
    showlegend=False,
    margin=dict(t=80, b=80, l=60, r=60)  # Add margins around the entire figure
)

# Update axes with better formatting and spacing
# Row 1: Scatter plots
fig3.update_xaxes(title_text="Years of Experience", title_font_size=12, row=1, col=1)
fig3.update_xaxes(title_text="Years of Experience", title_font_size=12, row=1, col=2)
fig3.update_yaxes(title_text="Salary ($)", title_font_size=12, row=1, col=1)
fig3.update_yaxes(title_text="Salary ($)", title_font_size=12, row=1, col=2)

# Row 2: Histograms
fig3.update_xaxes(title_text="Salary ($)", title_font_size=12, tickangle=0, row=2, col=1)
fig3.update_xaxes(title_text="Salary ($)", title_font_size=12, tickangle=0, row=2, col=2)
fig3.update_yaxes(title_text="Frequency", title_font_size=12, row=2, col=1)  # Changed to "Frequency" for clarity
fig3.update_yaxes(title_text="Frequency", title_font_size=12, row=2, col=2)

# Improve subplot title formatting
fig3.update_annotations(font_size=13, font_weight="bold")

fig3.update_layout(
    # Enhanced interactivity
    dragmode='pan',
    selectdirection='d',

    # Better mode bar with all export options
    modebar_add=[
        'zoomIn2d', 'zoomOut2d', 'pan2d', 'resetScale2d',
        'toImage', 'downloadImage', 'toggleSpikelines'
    ],
    modebar_remove=['lasso2d', 'select2d', 'autoScale2d'],
    modebar_orientation='v',
    modebar_bgcolor='rgba(255,255,255,0.9)',
    modebar_color='#2C3E50',

    # Enhanced hover
    hoverlabel=dict(
        bgcolor="white",
        font_size=11,
        font_family="Arial, sans-serif",
        bordercolor="navy"
    ),

    # Responsive design
    autosize=True,

    # Mobile responsiveness
    margin=dict(l=50, r=50, t=80, b=80, pad=4),

    # Mobile-friendly font sizes
    font=dict(
        family="Arial, sans-serif",
        size=11,
        color="#2C3E50"
    )
)

display_figure(fig3, "03_education_level_analysis")

The scatter plots demonstrate that advanced degree holders consistently achieve higher salaries across all experience levels, with the compensation gap expanding significantly with increased tenure. The salary distribution histograms reveal that Master's and PhD holders exhibit higher median salaries and greater earning potential compared to Bachelor's degree recipients.

This comprehensive analysis of education's impact on compensation reveals several key patterns. Advanced degree holders maintain a consistent salary premium throughout their careers, with the gap widening as experience increases. This suggests that higher education provides not just an initial salary boost, but also enhanced career progression and earning potential over time.

The histograms show that while Bachelor's degree holders form the largest group, advanced degree recipients are positioned in higher salary brackets, indicating the market's valuation of specialized knowledge and advanced credentials. This pattern has significant implications for educational investment decisions and career planning strategies.

In [ ]:
#| label: fig-experience-vs-salary-education
#| fig-cap: "Experience vs Salary by Education Group. Scatter plot of years of experience (with jitter) vs salary, colored by education grouping (Associate or Lower, Bachelor, Master's, PhD)."
# Build dataset for experience vs salary by education group (compute avg salary safely)
exp_spark = df.select(
    "MIN_EDULEVELS_NAME",
    col("MAX_YEARS_EXPERIENCE").cast("float").alias("MAX_YEARS_EXPERIENCE"),
    col("SALARY_FROM"),
    col("SALARY_TO")
).filter(
    (col("MAX_YEARS_EXPERIENCE").isNotNull()) &
    (col("MAX_YEARS_EXPERIENCE") > 0) &
    (col("MAX_YEARS_EXPERIENCE") <= 30)  # cap outliers
)

# Compute a robust average salary: prefer mean of SALARY_FROM and SALARY_TO when available
exp_spark = exp_spark.withColumn(
    "Average_Salary",
    when((col("SALARY_FROM").isNotNull()) & (col("SALARY_TO").isNotNull()) & (col("SALARY_FROM")>0) & (col("SALARY_TO")>0),
         (col("SALARY_FROM") + col("SALARY_TO")) / 2.0)
    .when((col("SALARY_FROM").isNotNull()) & (col("SALARY_FROM")>0), col("SALARY_FROM"))
    .when((col("SALARY_TO").isNotNull()) & (col("SALARY_TO")>0), col("SALARY_TO"))
    .otherwise(None)
)

# Filter to valid salary rows
exp_spark = exp_spark.filter((col("Average_Salary").isNotNull()) & (col("Average_Salary") > 0))

# Create explicit education groups using regex (case-insensitive)

# More comprehensive regexes to capture common variants (case-insensitive)
lower_pattern = "(?i)High school|Some college|Certificate|Associate|GED|No education listed|Secondary"
bachelor_pattern = "(?i)Bachelor|Bachelors|BA|BS"
higher_pattern = "(?i)Master|Masters|Professional|MSc|MA|MS"
# include dotted and spaced variants like 'Ph.D.' and 'Ph.D. or professional degree'
phd_pattern = "(?i)Doctoral|Doctorate|PhD|Ph\.D\.|DPhil|Doctor of|MD|M\.D\.|Doctor|Ph\.D"

exp_spark = exp_spark.withColumn(
    "EDU_GROUP",
    when(col("MIN_EDULEVELS_NAME").rlike(phd_pattern), "PhD")
    .when(col("MIN_EDULEVELS_NAME").rlike(higher_pattern), "Master's")
    .when(col("MIN_EDULEVELS_NAME").rlike(bachelor_pattern), "Bachelor")
    .when(col("MIN_EDULEVELS_NAME").rlike(lower_pattern), "Associate or Lower")
    .otherwise("Other")
)

# Bucket experience to integer years (so points cluster at integer x positions like your target figure)
exp_spark = exp_spark.withColumn("Experience_Years", floor(col("MAX_YEARS_EXPERIENCE")).cast("int"))

# Keep only the 4 main education groups for the plot
exp_spark = exp_spark.filter(col("EDU_GROUP").isin("Associate or Lower", "Bachelor", "Master's", "PhD"))

# Convert to pandas for plotting

exp_pd = exp_spark.select("MIN_EDULEVELS_NAME", "EDU_GROUP", "MAX_YEARS_EXPERIENCE", "Experience_Years", "Average_Salary").toPandas()
print(f"Records for experience vs salary by education: {len(exp_pd):,}")
# Debug: show counts per education group so we can confirm PhD mapping
print("EDU_GROUP counts:\n", exp_pd['EDU_GROUP'].value_counts(dropna=False))

# Add small jitter around the integer experience buckets to reduce overplotting (stddev 0.18)
np.random.seed(42)
exp_pd['experience_jitter'] = exp_pd['Experience_Years'] + np.random.normal(0, 0.18, len(exp_pd))

# Create scatter plot with Plotly Express using Average_Salary and EDU_GROUP
category_order = ["Associate or Lower", "Bachelor", "Master's", "PhD"]
color_map = {
    "Associate or Lower": "#636EFA",
    "Bachelor": "#EF553B",
    "Master's": "#00CC96",
    "PhD": "#AB63FA"
}

fig_exp = px.scatter(
    exp_pd,
    x='experience_jitter',
    y='Average_Salary',
    color='EDU_GROUP',
    category_orders={'EDU_GROUP': category_order},
    color_discrete_map=color_map,
    labels={'experience_jitter': 'Years of Experience', 'Average_Salary': 'Average Salary (USD)'},
    hover_data={'MIN_EDULEVELS_NAME': True, 'MAX_YEARS_EXPERIENCE': ':.1f', 'Average_Salary': ':.0f'},
    title='Experience vs Salary by Education Level'
)

fig_exp.update_traces(
    marker=dict(size=5, line=dict(width=0.15, color='DarkSlateGrey')), selector=dict(mode='markers')
)

fig_exp.update_layout(
    template='plotly_white',
    height=600,
    width=1400,
    margin=dict(t=80, b=60, l=80, r=80),
    legend=dict(title='Education Group', orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    xaxis=dict(title='Years of Experience', tickmode='linear', dtick=1),
    yaxis=dict(title='Average Salary (USD)', tickformat='$,.0f')
)

# Enhance and export
fig_exp = enhance_plotly_figure(fig_exp, "Experience vs Salary by Education Level", "06_experience_vs_salary_education", width=1400, height=600)

display_figure(fig_exp, "06_experience_vs_salary_education")

## Salary by Remote Work Type

In [ ]:
#| label: fig-remote-work
#| fig-cap: "Interactive scatter plot analysis of salary distributions across different remote work arrangements."

# Quick raw counts from `df` for debugging (counts are from the original Spark DataFrame before later filters)
raw_remote_counts = (
        df.select('REMOTE_TYPE_NAME')
            .na.fill('[None]', subset=['REMOTE_TYPE_NAME'])
            .groupBy('REMOTE_TYPE_NAME')
            .count()
            .orderBy(col('count').desc())
)
print('Top REMOTE_TYPE_NAME values (raw df):')
raw_remote_counts.show(40, truncate=False)

# Also show total counts per categorized group (Remote/Hybrid/Onsite) from raw df
def categorize_remote_spark(col_expr):
    # Prefer Hybrid when both words appear (e.g., 'Hybrid Remote') — check Hybrid first
    return when((col_expr.isNull()) | (col_expr == '') | (col_expr == '[None]'), 'Onsite') \
        .when(col_expr.rlike('(?i)Hybrid'), 'Hybrid') \
        .when(col_expr.rlike('(?i)Remote'), 'Remote') \
        .otherwise('Onsite')

cat_counts = (
        df.select(categorize_remote_spark(col('REMOTE_TYPE_NAME')).alias('remote_cat'))
            .groupBy('remote_cat')
            .count()
)
print('\nCategorized counts from raw df:')
cat_counts.show(truncate=False)

# Split into remote work groups as specified

remote_spark = df.select(
    "REMOTE_TYPE_NAME",
    "MAX_YEARS_EXPERIENCE",
    col("SALARY_FROM"),
    col("SALARY_TO"),
    "LOT_V6_SPECIALIZED_OCCUPATION_NAME"
).filter(
    (col("MAX_YEARS_EXPERIENCE").isNotNull()) &
    ((col("SALARY_FROM").isNotNull()) | (col("SALARY_TO").isNotNull())) &
    ((col("SALARY_FROM") > 0) | (col("SALARY_TO") > 0)) &
    (col("MAX_YEARS_EXPERIENCE") <= 30) &
    (col("LOT_V6_SPECIALIZED_OCCUPATION_NAME").isNotNull())
)

# Convert to Pandas
remote_pd = remote_spark.toPandas()

print(f"Records for remote work analysis: {len(remote_pd):,}")

# Create remote work categories as specified in assignment
def categorize_remote(remote_type):
    if pd.isna(remote_type) or remote_type == '[None]' or remote_type == '' or remote_type is None:
        return 'Onsite'
    # Prefer 'Hybrid' when both words appear (e.g., 'Hybrid Remote')
    elif 'Hybrid' in str(remote_type):
        return 'Hybrid'
    elif 'Remote' in str(remote_type):
        return 'Remote'
    else:
        return 'Onsite'

remote_pd['remote_category'] = remote_pd['REMOTE_TYPE_NAME'].apply(categorize_remote)

# Print distribution
print("Remote work distribution:")
print(remote_pd['remote_category'].value_counts())

# Create subplot structure with better spacing
fig4 = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        "Remote - Experience vs Salary",
        "Hybrid - Experience vs Salary",
        "Onsite - Experience vs Salary",
        "Remote - Salary Distribution",
        "Hybrid - Salary Distribution",
        "Onsite - Salary Distribution"
    ),
    specs=[[{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]],
    horizontal_spacing=0.12,  # Add horizontal spacing
    vertical_spacing=0.15     # Add vertical spacing
)

# Add jitter as specified
np.random.seed(42)
remote_pd = remote_pd.copy()
# Cast experience to float and add a smaller jitter (stddev=0.18) so points cluster by integer years
remote_pd['MAX_YEARS_EXPERIENCE'] = remote_pd['MAX_YEARS_EXPERIENCE'].astype(float)
remote_pd = remote_pd[remote_pd['MAX_YEARS_EXPERIENCE'] > 0]
remote_pd['experience_years'] = np.floor(remote_pd['MAX_YEARS_EXPERIENCE']).astype(int)
remote_pd['experience_jitter'] = remote_pd['experience_years'] + np.random.normal(-0.2, 0.2, len(remote_pd))

# Compute Average_Salary (prefer mean of SALARY_FROM & SALARY_TO)
def compute_avg_salary(row):
    f = row.get('SALARY_FROM')
    t = row.get('SALARY_TO')
    if pd.notna(f) and pd.notna(t) and f > 0 and t > 0:
        return (f + t) / 2.0
    if pd.notna(f) and f > 0:
        return f
    if pd.notna(t) and t > 0:
        return t
    return None

remote_pd['Average_Salary'] = remote_pd.apply(compute_avg_salary, axis=1)
remote_pd = remote_pd[remote_pd['Average_Salary'].notna() & (remote_pd['Average_Salary'] > 0)]

# Define colors for each remote type
colors = {'Remote': '#27AE60', 'Hybrid': '#F39C12', 'Onsite': '#8E44AD'}

# Create scatter plots and histograms for each remote type
for i, remote_type in enumerate(['Remote', 'Hybrid', 'Onsite']):
    data = remote_pd[remote_pd['remote_category'] == remote_type]

    if len(data) > 0:
        print(f"{remote_type}: {len(data):,} records")
        # Add small vertical jitter to Average_Salary for plotting (-500..+500 uniform)
        y_jitter = np.random.uniform(-500, 500, len(data))
        data_plot_y = data['Average_Salary'].values + y_jitter

        # Scatter plot with jitter (using go.Scatter). Keep the hover showing the true Average_Salary
        fig4.add_trace(
            go.Scatter(
                x=data['experience_jitter'],
                y=data_plot_y,
                mode='markers',
                name=f"{remote_type}",
                marker=dict(color=colors[remote_type], size=5, opacity=0.75, line=dict(width=0.15, color='DarkSlateGrey')),
                hovertemplate=f'{remote_type}<br>Occupation: %{{customdata[0]}}<br>Experience: %{{x:.1f}} years<br>Salary: $%{{customdata[1]:,.0f}}<extra></extra>',
                customdata=np.stack([data['LOT_V6_SPECIALIZED_OCCUPATION_NAME'], data['Average_Salary']], axis=-1),
                showlegend=False
            ),
            row=1, col=i+1
        )

        # Histogram
        fig4.add_trace(
            go.Histogram(
                x=data['Average_Salary'],
                name=f"{remote_type} Distribution",
                marker_color=colors[remote_type],
                opacity=0.7,
                nbinsx=25,
                showlegend=False
            ),
            row=2, col=i+1
        )

# Update layout with better spacing and readability
fig4.update_layout(
    title={
        'text': "Salary Analysis by Remote Work Type",
        'x': 0.5,
        'font': {'size': 18, 'family': 'Arial, sans-serif', 'color': '#2C3E50'}
    },
    template="plotly_white",
    font_family="Arial, sans-serif",
    plot_bgcolor="white",
    paper_bgcolor="white",
    height=900,  # Increased height for better spacing
    margin=dict(l=60, r=60, t=100, b=80, pad=10),  # Better margins
    showlegend=False  # Remove legend to save space
)

# Update axes labels with better formatting
for i in range(1,4):
    fig4.update_xaxes(title_text='Years of Experience', row=1, col=i, tickmode='linear', dtick=1)
    fig4.update_xaxes(title_text='Salary (USD)', row=2, col=i)
    fig4.update_yaxes(title_text='Average Salary (USD)', row=1, col=i, tickformat='$,.0f')

# Enhance and export
fig4 = enhance_plotly_figure(fig4, "Salary by Remote Work Type", "07_salary_by_remote_work_type", width=1400, height=900)
display_figure(fig4, "07_salary_by_remote_work_type")

# Short descriptions / observations
print('\nObservations:')
for remote_type in ['Remote','Hybrid','Onsite']:
    subset = remote_pd[remote_pd['remote_category']==remote_type]
    if len(subset)==0:
        print(f"{remote_type}: no data")
        continue
    med = subset['Average_Salary'].median()
    mean = subset['Average_Salary'].mean()
    cnt = len(subset)
    print(f"{remote_type}: n={cnt:,}, median=${med:,.0f}, mean=${mean:,.0f}.")
    if remote_type=='Remote':
        print("  Remote roles show a modest salary premium on average compared with Onsite, visible as higher median and more points in upper salary ranges.")
    if remote_type=='Hybrid':
        print("  Hybrid roles fall between Remote and Onsite, often showing moderate salaries with similar spread.")
    if remote_type=='Onsite':
        print("  Onsite roles have a wider concentration at lower salary bands, though senior onsite roles still reach high salaries.")
for i in range(1, 4):
    # Scatter plot axes (top row)
    fig4.update_xaxes(
        title_text="Years of Experience",
        row=1, col=i,
        title_font=dict(size=12),
        tickfont=dict(size=10),
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)'
    )
    fig4.update_yaxes(
        title_text="Salary ($)",
        row=1, col=i,
        title_font=dict(size=12),
        tickfont=dict(size=10),
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        tickformat='$,.0f'
    )

    # Histogram axes (bottom row)
    fig4.update_xaxes(
        title_text="Salary ($)",
        row=2, col=i,
        title_font=dict(size=12),
        tickfont=dict(size=10),
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)',
        tickformat='$,.0f'
    )
    fig4.update_yaxes(
        title_text="Count",
        row=2, col=i,
        title_font=dict(size=12),
        tickfont=dict(size=10),
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)'
    )

# Enhance with interactive features and export capabilities
fig4 = enhance_plotly_figure(fig4, "Salary Analysis by Remote Work Type",
                           "04_remote_work_analysis", width=1400, height=900)

display_figure(fig4, "04_remote_work_analysis")

print('\n--- Remote group summaries (top-level) ---')

if len(remote_pd) == 0:
    print('remote_pd is empty (no rows after filtering)')
else:
    group_stats = remote_pd.groupby('remote_category').agg(
        count=('Average_Salary','size'),
        median_salary=('Average_Salary','median'),
        mean_salary=('Average_Salary','mean')
    ).reset_index()
    print(group_stats.to_string(index=False))

for cat in ['Remote','Hybrid','Onsite']:
    subset = remote_pd[remote_pd['remote_category']==cat]
    print(f"\nSample rows for {cat} (up to 5): {len(subset):,} rows total")
    if len(subset)>0:
        try:
            display(subset[['REMOTE_TYPE_NAME','LOT_V6_SPECIALIZED_OCCUPATION_NAME','MAX_YEARS_EXPERIENCE','experience_years','Average_Salary']].head(5))
        except Exception:
            print(subset[['REMOTE_TYPE_NAME','LOT_V6_SPECIALIZED_OCCUPATION_NAME','MAX_YEARS_EXPERIENCE','experience_years','Average_Salary']].head(5).to_string(index=False))
    else:
        print('  (no rows)')

print('\n--- Education group summaries (top-level) ---')
if 'exp_pd' in globals() and isinstance(exp_pd, pd.DataFrame):
    edu_stats = exp_pd.groupby('EDU_GROUP').agg(
        count=('Average_Salary','size'),
        median_salary=('Average_Salary','median')
    ).reset_index()
    print(edu_stats.to_string(index=False))
    for g in exp_pd['EDU_GROUP'].unique():
        sub = exp_pd[exp_pd['EDU_GROUP']==g]
        print(f"\nSample rows for EDU_GROUP={g} (up to 5): {len(sub):,} rows total")
        try:
            display(sub[['MIN_EDULEVELS_NAME','MAX_YEARS_EXPERIENCE','Experience_Years','Average_Salary']].head(5))
        except Exception:
            print(sub[['MIN_EDULEVELS_NAME','MAX_YEARS_EXPERIENCE','Experience_Years','Average_Salary']].head(5).to_string(index=False))
else:
    print('exp_pd not available for education debug (it may be in a different scope).')

# Summary statistics by remote type (original)
print("\nSalary summary by remote work type:")
for remote_type in ['Remote', 'Hybrid', 'Onsite']:
    data = remote_pd[remote_pd['remote_category'] == remote_type]['Average_Salary']
    if len(data) > 0:
        print(f"{remote_type}: Mean=${data.mean():,.0f}, Median=${data.median():,.0f}, Count={len(data):,}")

# Combined scatter: MAX_YEARS_EXPERIENCE vs Average_Salary colored by remote group
# Use a small horizontal jitter on experience and a small vertical jitter on salary (-500..500)
print('\nBuilding combined scatter: MAX_YEARS_EXPERIENCE vs Average_Salary (colored by remote group)')
combined_pd = remote_pd.copy()
combined_pd['MAX_YEARS_EXPERIENCE'] = combined_pd['MAX_YEARS_EXPERIENCE'].astype(float)
combined_pd = combined_pd[combined_pd['MAX_YEARS_EXPERIENCE'] > 0]

np.random.seed(42)
combined_pd['x_jitter'] = combined_pd['MAX_YEARS_EXPERIENCE'] + np.random.normal(0, 0.12, len(combined_pd))
combined_pd['y_jitter'] = combined_pd['Average_Salary'] + np.random.uniform(-500, 500, len(combined_pd))

fig_comb = px.scatter(
    combined_pd,
    x='x_jitter',
    y='y_jitter',
    color='remote_category',
    color_discrete_map={'Remote': '#27AE60', 'Hybrid': '#F39C12', 'Onsite': '#8E44AD'},
    labels={'x_jitter': 'Years of Experience (jittered)', 'y_jitter': 'Average Salary (USD)', 'remote_category': 'Remote Work Type'},
    hover_data={'LOT_V6_SPECIALIZED_OCCUPATION_NAME': True, 'MAX_YEARS_EXPERIENCE': ':.1f', 'Average_Salary': ':.0f'},
    opacity=0.75,
    height=600,
    width=1400
)

fig_comb.update_traces(marker=dict(size=5, line=dict(width=0.15, color='DarkSlateGrey')))
fig_comb.update_layout(title='Experience vs Average Salary by Remote Work Type', xaxis_title='Years of Experience', yaxis_title='Average Salary (USD)', xaxis=dict(tickmode='linear', dtick=1), yaxis=dict(tickformat='$,.0f'))

# Export and display
fig_comb = enhance_plotly_figure(fig_comb, 'Experience vs Average Salary by Remote Work Type (combined)', '07b_experience_vs_salary_by_remote', width=1400, height=600)
display_figure(fig_comb, '07b_experience_vs_salary_by_remote')

# Short summary for the combined scatter
print('\nCombined scatter summary:')
for rc in ['Remote','Hybrid','Onsite']:
    sub = combined_pd[combined_pd['remote_category']==rc]
    if len(sub)==0:
        print(f"{rc}: no rows after filtering")
        continue
    print(f"{rc}: n={len(sub):,}, median=${sub['Average_Salary'].median():,.0f}, mean=${sub['Average_Salary'].mean():,.0f}")

Remote work positions demonstrate the highest salary potential with considerable variance, indicating premium compensation for distributed work capabilities. Hybrid arrangements provide competitive salaries while maintaining work flexibility, whereas onsite positions cluster around lower median compensation levels.

The analysis reveals a clear hierarchy in remote work compensation, with fully remote positions commanding the highest salaries. This premium likely reflects the specialized skills required for effective remote collaboration and the market's recognition of the value of location-independent work capabilities.

Hybrid arrangements offer a balanced approach, providing competitive compensation while maintaining some degree of workplace interaction. Onsite positions, while essential for many roles, generally offer lower compensation levels, reflecting the traditional employment model's baseline expectations.

These findings have significant implications for work arrangement preferences and compensation negotiations in an increasingly flexible labor market.